# Implmenting an ETL pipeline and Data warehouse using AWS services like S3, Glue, Athena and Redshift
- I will be using the same dataset I used for creating the etl pipeline and data warehouse using Python and Postgresql
- In this notebook, I will be using boto3 which is AWS SDK for python to connect to aws account and access the AWS services.
- Here, I am trying to practice Infrastructure as a Code(IaaC) in this project.

## Main objective
- To implement ETL pipeline and Data warehouse using AWS Services.
- To learn and understand about AWS services

### Importing Necessary libraries

In [34]:
# To connect to AWS 
import boto3
# To parse the data in config file
import configparser
# to process data
import pandas as pd 
# using json
import json
# to connect to Redshift cluster
import psycopg2

### Configuring AWS Account to be used from local jupyter notebook
- I have created a configuration file dwh.cfg which includes all the necessary configurations for connecting to AWS account

In [2]:
# Initialize the configparser object
config = configparser.ConfigParser()
# read the configuration from local file
config.read_file(open('dwh.cfg'))
# Load the parameters from the config file into variables
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
AWS_REGION             = config.get("AWS","DEFAULT_REGION")            

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

# * Checkout the parameters in a pandas dataframe
# pd.DataFrame({"Param":
#                   ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
#               "Value":
#                   [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
#              })

### Creating boto3 clients for S3, IAM, Glue and Redshift
These clients will then be used to access those AWS services and perform various services with those services

In [3]:
try:
    # ec2 is needed for creating and configuring security group for redshift
    ec2 = boto3.client('ec2',
                        region_name=AWS_REGION,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)
    
    s3 = boto3.client('s3',
                        region_name=AWS_REGION,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)

    iam = boto3.client('iam',
                        region_name=AWS_REGION,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)

    glue = boto3.client('glue',
                        region_name=AWS_REGION,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)
    
    athena = boto3.client('athena',
                    region_name=AWS_REGION,
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

    redshift = boto3.client('redshift',
                            region_name=AWS_REGION,
                            aws_access_key_id=KEY,
                            aws_secret_access_key=SECRET)
    
    print("Boto3 clients created succesfully")
except Exception as e:
    print("Folowing error was encountered:\n{e}")

Boto3 clients created succesfully


### Transforming the data from csv to parquet format before storing it in S3 bucket
- Parquet format is chosen as it optimizes query performance by allowing for efficient column pruning and data skipping. Parquet stores data in columnar format and also uses compression algorithm that reduces storage costs and also improve query performance
- To transform data from csv to parquet we use pandas

In [11]:
# Read the csv dataset
# Dataset Folder
DATASET_FOLDER = 'datasets'

df1 = pd.read_csv(f"{DATASET_FOLDER}/product_info.csv")
# low_memory=False to deal with mixed datatypes warning
df2 = pd.read_csv(f"{DATASET_FOLDER}/product_reviews.csv",index_col=0,low_memory=False)

# Transform to parquet format
df1_t = df1.to_parquet(f"{DATASET_FOLDER}/product_info.parquet")
df2_t = df2.to_parquet(f"{DATASET_FOLDER}/product_reviews.parquet")
print("Data succesfully converted to parquet format")

Data succesfully converted to parquet format


### Using S3 service to upload the dataset to S3 bucket
- List all existing buckets in aws account
- Check if the required s3 bucket exists or not.
- If not create a new bucket.
- Upload the file from local directory to S3 bucket
> Note: The S3 bucket name should be globally unique else you will get error

In [70]:
SOURCE_DATA_BUCKET="coderush-anish-etl-source-data"

In [71]:
# function to upload files to s3 bucket
def upload_files_to_s3_bucket(files_to_upload: list):
    # Uploading the dataset files from local directory to s3 bucket
    try:
        print("Started uploading the files . . .")
        # print(files_to_upload)
        for file_name in files_to_upload:
            s3.upload_file(f"{DATASET_FOLDER}/{file_name}",SOURCE_DATA_BUCKET,file_name)
        print('Successfully uploaded all files')
    except Exception as e: 
        print(f"Failed to upload datasets to s3 bucket.\nFollowing error occured:\n{e}")

In [72]:
SOURCE_DATA_BUCKET="coderush-anish-etl-source-data"

try:
    # get list of all available buckets
    response = s3.list_buckets()
    # print(response) 
    #? Response looks like this
    # {'ResponseMetadata': {'RequestId': 'YMYR3MA9HN636GKE', 'HostId': 'GdfIlZp7Pd2k1SGdSHeLzPac01j/HoJRtilc/m3h5HMqjfBvZ9wqbIU3fIAObcJ6Xjc2HNVMJ4s=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'GdfIlZp7Pd2k1SGdSHeLzPac01j/HoJRtilc/m3h5HMqjfBvZ9wqbIU3fIAObcJ6Xjc2HNVMJ4s=', 'x-amz-request-id': 'YMYR3MA9HN636GKE', 'date': 'Sun, 14 May 2023 08:09:14 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'anish-shilpakar-ccp-2022-demo', 'CreationDate': datetime.datetime(2023, 1, 16, 9, 40, 15, tzinfo=tzutc())}, {'Name': 'anish-shilpakar-replication-demo', 'CreationDate': datetime.datetime(2023, 1, 16, 9, 28, 12, tzinfo=tzutc())}, {'Name': 'anish-shilpakar-server-access-logs', 'CreationDate': datetime.datetime(2023, 1, 16, 9, 12, 36, tzinfo=tzutc())}, {'Name': 'coderush-anish-etl-source-data', 'CreationDate': datetime.datetime(2023, 5, 14, 7, 39, 17, tzinfo=tzutc())}, {'Name': 'elasticbeanstalk-ap-south-1-977481651193', 'CreationDate': datetime.datetime(2023, 1, 17, 12, 31, 59, tzinfo=tzutc())}], 'Owner': {'ID': 'ba72a4b013b258edfa3487309159137f7a6ebb75e63e86982391186c8ca1bf29'}}
    buckets_list = []
    for res in response['Buckets']:
        buckets_list.append(res['Name'])
    # print(buckets_list)
except Exception as e:
    print(f"Error fetching the s3 buckets. Following error encountered:\n{e}")
    
# Create S3 bucket if not exists
if SOURCE_DATA_BUCKET not in buckets_list:
    print(f"Creating {SOURCE_DATA_BUCKET} . . .")
    try:
        response = s3.create_bucket(
            Bucket=SOURCE_DATA_BUCKET,
            CreateBucketConfiguration={
                    'LocationConstraint': 'ap-south-1'
                    }
        )
        print(f"{SOURCE_DATA_BUCKET} created successfully !")
    except Exception as e:
        print(f"Failed to create {SOURCE_DATA_BUCKET}. Following error encountered:\n{e}")
else:
    print(f"{SOURCE_DATA_BUCKET} already exists!")

# Uploading the dataset files from local directory to s3 bucket
upload_files_to_s3_bucket(["product_info.parquet","product_reviews.parquet"])

coderush-anish-etl-source-data already exists!
Started uploading the files . . .
Successfully uploaded all files


### Implementing Glue using Boto3 to perform ETL i-e Extract Step
1. Create glue database
2. Create Glue Crawler to automatically extract the schema of the dataset files to create Glue Datacatalog.
3. This should create tables for the data files in Glue in which the data is loaded.

In [31]:
DATABASE_NAME = "final_project_db"
# check if the database already exists or not
try:
    response = glue.get_databases()
    # print(response)
    # ? Response looks like this
    # {'DatabaseList': [{'Name': 'testdb', 'CreateTime': datetime.datetime(2023, 5, 14, 13, 52, 22, tzinfo=tzlocal()), 'CreateTableDefaultPermissions': [{'Principal': {'DataLakePrincipalIdentifier': 'IAM_ALLOWED_PRINCIPALS'}, 'Permissions': ['ALL']}], 'CatalogId': '977481651193'}], 'ResponseMetadata': {'RequestId': '0548b52d-3533-4c64-8a16-577a6b7e94aa', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 14 May 2023 08:07:26 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '249', 'connection': 'keep-alive', 'x-amzn-requestid': '0548b52d-3533-4c64-8a16-577a6b7e94aa'}, 'RetryAttempts': 0}}
    databases_list = []
    for db in response['DatabaseList']:
        databases_list.append(db['Name'])
    # print(databases_list)
except Exception as e:
    print(f"Failed to get the databases list.Following error occured:\n{e}")

if DATABASE_NAME not in databases_list:
# create database using boto3
    try:
        print(f"Started creating {DATABASE_NAME} . . .")
        response = glue.create_database(
            DatabaseInput={
                'Name': DATABASE_NAME,
            }
        )
        print(f"{DATABASE_NAME} created successfully")
    except Exception as e:
        print("Failed to created Glue database. Following error occured:\n{e}")
else: 
    print(f"{DATABASE_NAME} already exists")

final_project_db already exists


In [34]:
CRAWLER_NAME = "final_data_crawler"
ROLE_NAME="glue-crawler-role"
S3_PATH="s3://coderush-anish-etl-source-data/product_reviews.parquet"
# check if the crawler already exists in glue
try:
    response = glue.get_crawlers()
    # print(response)
    # ? Response looks like this
    # {'Crawlers': [{'Name': 'test-crawler', 'Role': 'glue-crawler-role', 'Targets': {'S3Targets': [{'Path': 's3://coderush-anish-etl-source-data/product_info.parquet', 'Exclusions': []}], 'JdbcTargets': [], 'MongoDBTargets': [], 'DynamoDBTargets': [], 'CatalogTargets': [], 'DeltaTargets': []}, 'DatabaseName': 'final_project_db', 'Classifiers': [], 'RecrawlPolicy': {'RecrawlBehavior': 'CRAWL_EVERYTHING'}, 'SchemaChangePolicy': {'UpdateBehavior': 'UPDATE_IN_DATABASE', 'DeleteBehavior': 'DEPRECATE_IN_DATABASE'}, 'LineageConfiguration': {'CrawlerLineageSettings': 'DISABLE'}, 'State': 'RUNNING', 'CrawlElapsedTime': 5770, 'CreationTime': datetime.datetime(2023, 5, 14, 18, 30, 35, tzinfo=tzlocal()), 'LastUpdated': datetime.datetime(2023, 5, 14, 18, 30, 35, tzinfo=tzlocal()), 'Version': 1, 'Configuration': '{"Version":1.0,"CreatePartitionIndex":true}', 'LakeFormationConfiguration': {'UseLakeFormationCredentials': False, 'AccountId': ''}}], 'ResponseMetadata': {'RequestId': 'd0c90f61-4f17-49d5-b106-4be2e923a11a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 14 May 2023 12:45:44 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '861', 'connection': 'keep-alive', 'x-amzn-requestid': 'd0c90f61-4f17-49d5-b106-4be2e923a11a'}, 'RetryAttempts': 0}}
    crawlers_list = []
    for crawler in response['Crawlers']:
        crawlers_list.append(crawler['Name'])
    # print(crawlers_list)
except Exception as e:
    print(f"Failed to get the crawlers list.Following error occured:\n{e}")

# check if the crawler already exists before creating the cluster
if CRAWLER_NAME not in crawlers_list:
    try:
        response = glue.create_crawler(
            Name=CRAWLER_NAME,
            Role=ROLE_NAME,
            DatabaseName=DATABASE_NAME,
            Description="AWS Glue crawler to crawl the source data",
            Targets={
                'S3Targets': [
                    {
                        'Path': S3_PATH,
                        'Exclusions': [
                            'string',
                        ]
                    },
                ]
            }
        )
        print(f"Successfully created crawler {CRAWLER_NAME}")
    except Exception as e:
        print(f"Failed to create the crawler {CRAWLER_NAME}\nFollowing error occured:\n{e}")

Successfully created crawler final_data_crawler


In [35]:
# Start running crawler
try:
    response = glue.start_crawler(
        Name=CRAWLER_NAME
    )
    # print(response)
    print(f"Successfully started running Glue crawler {CRAWLER_NAME}")
except Exception as e:
    print(f"Error when starting crawler.\nFollowing error occured:\n{e}")

{'ResponseMetadata': {'RequestId': 'f5387ba3-ffd5-44c8-9f3a-80ef78cf0570', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 14 May 2023 12:56:56 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'f5387ba3-ffd5-44c8-9f3a-80ef78cf0570'}, 'RetryAttempts': 0}}


ERROR: An issue I am facing, don't know if it is an issue of glue or issue of dataset, but I can create glue crawler and then create tables using that crawler but after that when I query the table using Athena only the column names are visible and there is no data loaded inside that table.   

So for now I will be doing the transform step using pandas

### Query the Glue tables using Amazon Athena

In [37]:
S3_STAGING_DIR = "s3://anish-shilpakar-athena-query-results/output/"
response = athena.start_query_execution(
    QueryString = "SELECT * FROM product_info_parquet",
    QueryExecutionContext = {"Database":DATABASE_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)
print(response)

{'QueryExecutionId': '92456b91-cfc3-49f4-82ce-abf2d315a7cd', 'ResponseMetadata': {'RequestId': 'ebd42f7e-6524-437c-9829-f4855efd2910', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 14 May 2023 13:09:52 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'ebd42f7e-6524-437c-9829-f4855efd2910'}, 'RetryAttempts': 0}}


In [38]:
import time
S3_BUCKET_NAME="anish-shilpakar-athena-query-results"
S3_OUTPUT_DIRECTORY="output"
Dict = {}
def download_and_load_query_results(client: boto3.client, query_response: Dict) -> pd.DataFrame:
    while True:
        try:
            #This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)


In [39]:
athena_results = download_and_load_query_results(athena,response)
athena_results.head()

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price


As you can see for some reason data is not being loaded in the glue tables even after running the glue crawler

### Extract and Transform Steps
### So instead using Pandas to transform the data

In [40]:
dataset1_path = "datasets/product_info.csv"
dataset2_path = "datasets/product_reviews.csv"

In [46]:
# This function will check whether the value is a numeric value or not
def checkNumeric(val):
    return True if str(val).isdigit() else False

In [41]:
def extract_data():
    print("Starting Extract Step . . .")
    df1 = pd.read_csv(dataset1_path)
    df2 = pd.read_csv(dataset2_path,index_col=0)
    # considering only the required columns from both dataframes
    df1 = df1[['product_id', 'brand_id', 'loves_count','rating', 'reviews','primary_category','child_count']]
    df2 = df2[['author_id', 'rating', 'submission_time', 'review_text',
        'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
        'product_id', 'product_name', 'brand_name', 'price_usd']]
    print("Extract Step Completed!!!")
    return df1, df2 

In [78]:
def transform_data(df1,df2):
    print("Starting Transform Step . . .")
    # merge two datasets based on product_id 
    df_merged = pd.merge(df1,df2,how="inner",on="product_id")
    # Rename Columns 
    rename_dict = {
    "loves_count": "favorites_count",
    "rating_x": "avg_product_rating",
    "reviews": "product_reviews_count",
    "primary_category": "product_category",
    "child_count": "variations_count",
    "rating_y": "review_rating",
    "submission_time": "full_date",
    "skin_tone": "reviewer_skin_tone",
    "skin_type": "reviewer_skin_type",
    "eye_color": "reviewer_eye_color",
    "hair_color": "reviewer_hair_color",
    "price_usd": "product_price",
    "author_id": "reviewer_id"
    }
    df_merged = df_merged.rename(columns=rename_dict)
    # cleaning nan values
    # 1. drop rows with nan values in review_text as it is most essential for product_reviews
    df_merged = df_merged.dropna(subset=["review_text"])
    #2. review_title is optional, so for rows with nan in review_title but certain values in review_text, fill the nan values with a default placeholder value
    df_merged["review_title"] = df_merged["review_title"].fillna("Review Provided")
    # 3. For categorical columns like reviewer_skin_tone, reviewer_skin_type, reviewer_hair_color, reviewer_eye_color replace the nan values with mode
    df_merged["reviewer_skin_tone"] = df_merged["reviewer_skin_tone"].fillna(df_merged["reviewer_skin_tone"].mode()[0])
    df_merged["reviewer_eye_color"] = df_merged["reviewer_eye_color"].fillna(df_merged["reviewer_eye_color"].mode()[0])
    df_merged["reviewer_skin_type"] = df_merged["reviewer_skin_type"].fillna(df_merged["reviewer_skin_type"].mode()[0])
    df_merged["reviewer_hair_color"] = df_merged["reviewer_hair_color"].fillna(df_merged["reviewer_hair_color"].mode()[0])
    # Creating new columns for date 
    df_merged["full_date"] = pd.to_datetime(df_merged["full_date"])
    df_merged["year"] = df_merged["full_date"].dt.year
    df_merged["month"] = df_merged["full_date"].dt.month
    df_merged["day"] = df_merged["full_date"].dt.day
    # Converting the values in reviewer_id column to int from object
    # checking for non-numeric reviewer_id
    df_merged["is_numeric"] = df_merged["reviewer_id"].apply(checkNumeric)
    # removing non numeric reviewer ids
    df_merged = df_merged[df_merged["is_numeric"] == True]
    # dropping the is_numeric column
    df_merged.drop(columns=["is_numeric"],inplace=True)
    # converting reviewer_id to int
    df_merged["reviewer_id"] = df_merged["reviewer_id"].apply(lambda x: int(x))
    # Creating multiple dataframes for fact and dimension tables
    # Product Reviews Table: Fact Table
    reviews_df = df_merged[['product_id','brand_id','reviewer_id','full_date','review_title','review_text','review_rating']]
    reviews_df = reviews_df.rename(columns={'full_date': 'date_id'})
    reviews_df = reviews_df.reset_index(drop=True)
    reviews_df.insert(0, 'review_id', reviews_df.index + 1)
    # Product table: Dimension table
    product_df = df_merged[['product_id', 'product_name', 'avg_product_rating', 'product_price', 'product_reviews_count', 'favorites_count', 'variations_count', 'product_category']]
    # To keep only unique product descriptions in product_df
    product_df = product_df.drop_duplicates("product_id").reset_index(drop=True)
    # Brand table: dimension table
    brand_df = df_merged[['brand_id', 'brand_name']]
    # To keep only the unique brand details in brands dataframe
    brand_df = brand_df.drop_duplicates("brand_id").reset_index(drop=True)
    # Reviewer table: dimension table
    reviewer_df = df_merged[['reviewer_id', 'reviewer_skin_tone', 'reviewer_skin_type', 'reviewer_eye_color', 'reviewer_hair_color']]
    # To keep only unique reviewer details
    reviewer_df = reviewer_df.drop_duplicates("reviewer_id").reset_index(drop=True)
    # Date table: dimension table
    date_df = df_merged[['full_date', 'year', 'month', 'day']]
    date_df = date_df.rename(columns={"full_date": "date_id"})
    date_df = date_df.drop_duplicates("date_id").reset_index(drop=True)
    print("Transform Step Completed")
    # Return the dataframes for fact and dimension tables
    result = {
        "reviews": reviews_df,
        "product": product_df,
        "brand": brand_df,
        "reviewer": reviewer_df,
        "date": date_df
    }
    
    return result

In [66]:
def save_df_to_csvs(*args):
    for k,v in args[0].items():
        print(f"Saving {k}.csv . . .")
        v.to_csv(f"{DATASET_FOLDER}/{k}.csv",index=False)
    print("Files saved succesfully")

In [79]:
# ET using pandas
df1, df2 = extract_data()
result = transform_data(df1,df2)
save_df_to_csvs(result)

Starting Extract Step . . .


C:\Users\dell\AppData\Local\Temp\ipykernel_7804\3297522506.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(dataset2_path,index_col=0)


Extract Step Completed!!!
Starting Transform Step . . .
Transform Step Completed
Saving reviews.csv . . .
Saving product.csv . . .
Saving brand.csv . . .
Saving reviewer.csv . . .
Saving date.csv . . .
Files saved succesfully


### Finally loading the transformed csv files to redshift datawarehouse

#### Firstly uploading the transformed data files to s3

In [80]:
files_to_upload = [f"{name}.csv" for name in result.keys()]
upload_files_to_s3_bucket(files_to_upload)

Started uploading the files . . .
Successfully uploaded all files


#### Then Creating IAM role that makes Redshift able to access S3 bucket

In [82]:
# Create the IAM role
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
    RoleName = DWH_IAM_ROLE_NAME,
    Description = 'Allows Redshift cluster to call AWS service on your behalf.',
    AssumeRolePolicyDocument = json.dumps(
        {'Statement': [{'Action': 'sts:AssumeRole',
                       'Effect': 'Allow', 
                       'Principal': {'Service': 'redshift.amazonaws.com'}}],
         'Version': '2012-10-17'})
    )
    print(f'Role {DWH_IAM_ROLE_NAME} created successfully')
except Exception as e:
    print(f"Failure in creating new role {DWH_IAM_ROLE_NAME}.\nFollowing error occured:\n{e}")

1.1 Creating a new IAM Role
Role dwhRole created successfully


#### Attach Policy to the role

In [83]:
# Attach Policy
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                          PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                          )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

#### Getting ARN for IAM role

In [84]:
# Get and print the IAM role ARN
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::977481651193:role/dwhRole


#### Creating Redshift Cluster

In [90]:
try:
    print("Started creating Redshift cluster . . .")
    response = redshift.create_cluster(        
        # Hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        # Identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        # Role (to allow s3 access)
        IamRoles=[roleArn]
    )
    print(f"Successfully created Redshift cluster: {DWH_DB}")
except Exception as e:
    print(f"Failure when creating Redshift cluster {DWH_DB}\nFollowing error occured:\n{e}")

Started creating Redshift cluster . . .
Successfully created Redshift cluster: product_reviews_dw


#### Check the status of Redshift cluster
To continue loading data to the Redshift datawarehouse, the status of cluster should be changed to available

In [4]:
def checkRedshiftClusterStatus(props):
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
checkRedshiftClusterStatus(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,anish
4,DBName,product_reviews_dw
5,Endpoint,{'Address': 'dwhcluster.cyoryyfhsxbk.ap-south-...
6,VpcId,vpc-0a106004324381a9d
7,NumberOfNodes,4


#### Take Note of cluster endpint and role ARN

In [5]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.cyoryyfhsxbk.ap-south-1.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::977481651193:role/dwhRole


#### Open an incoming TCP port to access the cluster endpoint 
For this we create a new security group using ec2 client and add inbound rules with TCP port open for all ip addresses

In [10]:
# ec2 resource using boto3 to configure inbound rules for the security group
ec2_r = ec2 = boto3.resource('ec2', 
                   region_name=AWS_REGION,
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

In [67]:
# vpc = ec2_r.Vpc(id=myClusterProps['VpcId'])
# defaultSg = list(vpc.security_groups.all())[0]
# defaultSg.id

In [17]:


try:
    response = ec2.create_security_group(
    Description='Security group for redshift cluster',
    GroupName='redshift-sg'
    )
    # print(response)
    # ec2.SecurityGroup(id='sg-0435581179b364351')
    print("Security group created and configured")
    # Adding inbound rule to open an incoming TCP port to access the cluster endpoint
    response.authorize_ingress(
        GroupName=defaultSg.group_name,  
        CidrIp='0.0.0.0/0',  
        IpProtocol='TCP', 
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
    print("Inbound port configured successfully!")
except Exception as e:
    print(f"Failure when creating security group. Following error occured:\n{e}")

Security group created and configured
Inbound port configured successfully!


#### Change the security group of Redshift cluster

In [30]:
try:
    response = redshift.modify_cluster(
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
        VpcSecurityGroupIds=[
        defaultSg.id
        ]
    ) 
    print(response)
    print("Successfully modified the security group of redshift cluster")
except Exception as e:
    print(f"Failed to modify the security group of redshift cluster. Following error occured:\n{e}")

{'Cluster': {'ClusterIdentifier': 'dwhcluster', 'NodeType': 'dc2.large', 'ClusterStatus': 'available', 'ClusterAvailabilityStatus': 'Modifying', 'MasterUsername': 'anish', 'DBName': 'product_reviews_dw', 'Endpoint': {'Address': 'dwhcluster.cyoryyfhsxbk.ap-south-1.redshift.amazonaws.com', 'Port': 5439}, 'ClusterCreateTime': datetime.datetime(2023, 5, 14, 13, 53, 26, 872000, tzinfo=tzutc()), 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-03c149c7605c92e12', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'default', 'VpcId': 'vpc-0a106004324381a9d', 'AvailabilityZone': 'ap-south-1c', 'PreferredMaintenanceWindow': 'wed:06:00-wed:06:30', 'PendingModifiedValues': {}, 'ClusterVersion': '1.0', 'AllowVersionUpgrade': True, 'NumberOfNodes': 4, 'PubliclyAccessible': True, 'Encrypted

#### Connect to Redshift cluster using psycopg2

In [63]:
# Set the connection parameters
host = DWH_ENDPOINT
port = DWH_PORT
dbname = DWH_DB
user = DWH_DB_USER
password = DWH_DB_PASSWORD

# Connect to the Redshift cluster
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)
print("Connected succesfully to Redshift cluster")
# Open a cursor to perform database operations
cur = conn.cursor()

Connected succesfully to Redshift cluster


#### Creating tables for fact and dimension tables in Redshift cluster

In [38]:
CREATE_TABLE_QUERIES = [
    """ 
    CREATE TABLE IF NOT EXISTS tbl_product(
        product_id TEXT PRIMARY KEY,
        product_name TEXT,
        avg_product_rating FLOAT,
        product_price FLOAT,
        product_reviews_count FLOAT,
        favorites_count INT,
        variations_count INT,
        product_category TEXT
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS tbl_brand(
        brand_id INT PRIMARY KEY,
        brand_name TEXT UNIQUE
    )
    """,
    """ 
    CREATE TABLE IF NOT EXISTS tbl_reviewer(
        reviewer_id BIGINT PRIMARY KEY,
        reviewer_skin_tone TEXT,
        reviewer_skin_type TEXT,
        reviewer_eye_color TEXT,
        reviewer_hair_color TEXT
    )
    """,
    """ 
    CREATE TABLE IF NOT EXISTS tbl_date(
        date_id TIMESTAMP PRIMARY KEY,
        year INT,
        month INT,
        day INT
    )
    """,
    """ 
    CREATE TABLE IF NOT EXISTS tbl_product_reviews(
        review_id INT PRIMARY KEY,
        product_id TEXT REFERENCES tbl_product(product_id),
        brand_id INT REFERENCES tbl_brand(brand_id),
        reviewer_id BIGINT REFERENCES tbl_reviewer(reviewer_id),
        date_id TIMESTAMP REFERENCES tbl_date(date_id),
        review_title TEXT,
        review_text VARCHAR(512),
        review_rating INT
    )
    """
]


for i,query in enumerate(CREATE_TABLE_QUERIES):
    print(f"Creating table {i}")
    # Create a table
    cur.execute(query)

print("All tables created successfully")

# Commit the transaction
conn.commit()

Creating table 0
Creating table 1
Creating table 2
Creating table 3
Creating table 4
All tables created successfully


#### Load the data from S3 bucket to the tables in cluster
Here, we use COPY command to load the data from S3 bucket to the tables in redshift cluster

In [53]:
copy_query1 = """ 
COPY tbl_brand FROM 's3://coderush-anish-etl-source-data/brand.csv'
CREDENTIALS 'aws_iam_role=arn:aws:iam::977481651193:role/dwhRole' DELIMITER ',' REGION 'ap-south-1' IGNOREHEADER 1;
"""

cur.execute(copy_query1)
conn.commit()
print("Data inserted to brand table")

Data inserted to brand table


In [66]:
# queries to load the data from csv files stored in s3 bucket to the tables in datawarehouse
LOAD_DATA_QUERIES = [
    """ 
    COPY tbl_brand FROM 's3://coderush-anish-etl-source-data/brand.csv'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::977481651193:role/dwhRole' FORMAT CSV REGION 'ap-south-1' IGNOREHEADER 1;
    """,
    """ 
    COPY tbl_date FROM 's3://coderush-anish-etl-source-data/date.csv'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::977481651193:role/dwhRole' FORMAT CSV REGION 'ap-south-1' IGNOREHEADER 1;
    """,
    """ 
    COPY tbl_product FROM 's3://coderush-anish-etl-source-data/product.csv'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::977481651193:role/dwhRole' FORMAT CSV REGION 'ap-south-1' IGNOREHEADER 1;
    """,
    """ 
    COPY tbl_reviewer FROM 's3://coderush-anish-etl-source-data/reviewer.csv'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::977481651193:role/dwhRole' FORMAT CSV REGION 'ap-south-1' IGNOREHEADER 1;
    """,
    """ 
    COPY tbl_product_reviews FROM 's3://coderush-anish-etl-source-data/reviews.csv'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::977481651193:role/dwhRole' FORMAT CSV REGION 'ap-south-1' IGNOREHEADER 1;
    """
]

try:
    for i,query in enumerate(LOAD_DATA_QUERIES):
        print(f"Inserting data to table {i}")
        # Create a table
        cur.execute(query)

    print("Data inserted to all tables successfully")

    # Commit the transaction
    conn.commit()
except Exception as e:
    print(f"Failed to insert data to tables. Following error was encountered:\n{e}")
    conn.rollback()

Inserting data to table 0
Inserting data to table 1
Inserting data to table 2
Inserting data to table 3
Inserting data to table 4
Data inserted to all tables successfully


In [62]:
conn.close()

### Finally after loading the data into Redshift data warehouse data was queried to check if it has been properly loaded or not. The following output were obtained

### Outputs in Redshift Data warehouse
#### 1. Fact Table: Product Reviews
![Product Reviews](diagrams/product_reviews.jpeg)
#### 2. Dimension Table: Product
![Product](diagrams/product.jpeg)
#### 3. Dimension Table: Brand
![Brand](diagrams/brand.jpeg)
#### 4. Dimension Table: Reviewer
![Reviewer](diagrams/reviewer.jpeg)
#### 5. Dimension Table: Date
![Date](diagrams/date.jpeg)

ETL Completed using Python, Pandas and AWS Services : S3, RedShift